In [ ]:
from dask.distributed import Client
from pandas import DataFrame

import fastparquet as fp
import numpy as np
import pandas as pd

from colorcet import fire

from datashader.bundling import directly_connect_edges, hammer_bundle
from datashader.utils import export_image

import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
def render_points(df, width=4000, height=4000, cmap=["lightblue", "darkblue"], bgcolor=None):
    cvs = ds.Canvas(plot_width=width, plot_height=height, x_range=(0, 1), y_range=(0, 1))
    agg = cvs.points(df, 'x', 'y',  ds.count())
    img = tf.shade(agg, cmap=cmap)
    img = tf.spread(img)
    return tf.set_background(img, bgcolor) if bgcolor else img

In [ ]:
def render_lines(df, width=4000, height=4000, cmap=["lightblue", "darkblue"], bgcolor=None):
    cvs = ds.Canvas(plot_width=width, plot_height=height, x_range=(0, 1), y_range=(0, 1))
    agg = cvs.line(df, 'x', 'y',  ds.count())
    img = tf.shade(agg, cmap=cmap)
    return tf.set_background(img, bgcolor) if bgcolor else img

In [ ]:
client = Client()

### Edge bundling with graph read from Parquet

In [ ]:
researchers_nodes_df = fp.ParquetFile('data/calvert_uk_research2017_nodes.snappy.parq').to_pandas(index='id')
researchers_edges_df = fp.ParquetFile('data/calvert_uk_research2017_edges.snappy.parq').to_pandas(index='id')

In [ ]:
researchers_nodes_df.head()

In [ ]:
researchers_edges_df.head()

In [ ]:
researchers_direct_df = directly_connect_edges(researchers_nodes_df, researchers_edges_df)

#### Graph with only nodes

In [ ]:
render_points(researchers_nodes_df, width=2000, height=2000)

#### Graph without edge bundling

In [ ]:
render_lines(researchers_direct_df, width=2000, height=2000)

#### Graph with edge bundling

In [ ]:
%time researchers_df = hammer_bundle(researchers_nodes_df, researchers_edges_df, 0.05, 0.7)

In [ ]:
render_lines(researchers_df, width=2000, height=2000, cmap=fire, bgcolor='black')

### Edge bundling with random graph

In [ ]:
def generate_nodes(n):
    return pd.DataFrame(np.random.randn(n, 2), columns=['x', 'y'])

In [ ]:
def generate_edges(n, nodes):
    return pd.DataFrame(np.random.randint(len(nodes), size=(n, 2)), columns=['source', 'target'])

In [ ]:
def generate_random_graph(nodes, edges):
    ndf = generate_nodes(nodes)
    edf = generate_edges(edges, ndf)
    return ndf, edf

In [ ]:
random_nodes_df, random_edges_df = generate_random_graph(10000, 50000)

#### Graph with only nodes

In [ ]:
render_points(random_nodes_df, width=2000, height=2000)

#### Graph without edge bundling

In [ ]:
random_direct_df = directly_connect_edges(random_nodes_df, random_edges_df)
render_lines(random_direct_df, width=2000, height=2000)

#### Graph with edge bundling

In [ ]:
%time random_df = hammer_bundle(random_nodes_df, random_edges_df, 0.05, 0.7)

In [ ]:
render_lines(random_df, width=2000, height=2000, cmap=fire, bgcolor='black')

### Edge bundling with star graph and circular layout

In [ ]:
import networkx as nx

In [ ]:
graph = nx.star_graph(100000)

In [ ]:
layout = nx.circular_layout(graph)

In [ ]:
data = []
for node in graph.nodes_iter():
    x, y = layout[node]
    data.append([node, x, y])
circular_nodes_df = pd.DataFrame(data, columns=['id', 'x', 'y'])
circular_nodes_df.set_index('id', inplace=True)

In [ ]:
circular_edges_df = pd.DataFrame(graph.edges_iter(), columns=['source', 'target'])

#### Graph with only nodes

In [ ]:
render_points(circular_nodes_df, width=4000, height=4000)

#### Graph without edge bundling

In [ ]:
circular_direct_df = directly_connect_edges(circular_nodes_df, circular_edges_df)
render_lines(circular_direct_df, width=2000, height=2000)

#### Graph with edge bundling

In [ ]:
%time circular_df0 = hammer_bundle(circular_nodes_df, circular_edges_df, 0.05, 0.7)

In [ ]:
render_lines(circular_df0, width=2000, height=2000, cmap=fire, bgcolor='black')

#### Decrease decay for edge bundling

In [ ]:
# Decay from 0.7 to 0.4
%time circular_df1 = hammer_bundle(circular_nodes_df, circular_edges_df, initial_bandwidth=0.05, decay=0.4)

In [ ]:
render_lines(circular_df1, width=2000, height=2000, cmap=fire, bgcolor='black')

#### Increase decay for edge bundling

In [ ]:
# Decay from 0.7 to 1.0
%time circular_df2 = hammer_bundle(circular_nodes_df, circular_edges_df, initial_bandwidth=0.05, decay=1.0)

In [ ]:
render_lines(circular_df2, width=2000, height=2000, cmap=fire, bgcolor='black')

#### Decrease initial bandwidth for edge bundling

In [ ]:
# Initial bandwidth from 0.05 to 0.025
%time circular_df3 = hammer_bundle(circular_nodes_df, circular_edges_df, initial_bandwidth=0.025, decay=0.7)

In [ ]:
render_lines(circular_df3, width=2000, height=2000, cmap=fire, bgcolor='black')

#### Increase initial bandwidth for edge bundling

In [ ]:
# Initial bandwidth from 0.05 to 0.1
%time circular_df4 = hammer_bundle(circular_nodes_df, circular_edges_df, initial_bandwidth=0.1, decay=0.7)

In [ ]:
render_lines(circular_df4, width=2000, height=2000, cmap=fire, bgcolor='black')